---
title: Heart Failure Survival Analysis
author: "Merari Santana, Kevin Gao, Gurmehak Kaur, Yuhan Fan"
jupyter: python3
format:
  html:
    toc: true
    toc-depth: 2
    self-contained: true
bibliography: references.bib
execute:
    echo: false
---

In [ ]:
#| echo: false
#| output: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import altair_ally as aly
import os
from vega_datasets import data
from sklearn import set_config
from sklearn.model_selection import (GridSearchCV, cross_validate, train_test_split,)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandera as pa
from deepchecks.tabular.checks import FeatureLabelCorrelation, FeatureFeatureCorrelation
from deepchecks.tabular import Dataset
import warnings


# Enable Vegafusion for better data transformation
#aly.alt.data_transformers.enable('vegafusion')
#alt.data_transformers.enable('vegafusion')

In [ ]:
#| echo: false
#| output: false
#| label: tbl-model-metrics
#| tbl-cap: Evaluation metrics for the final model.

import pandas as pd

file_path = '../data/heart_failure_clinical_records_dataset.csv'
heart_failure_data = pd.read_csv(file_path)

# List of binary columns
binary_columns = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking', 'DEATH_EVENT']

# Convert all binary columns to True/False
heart_failure_data[binary_columns] = heart_failure_data[binary_columns].astype(bool)

heart_failure_data.shape

heart_failure_data.info()

heart_failure_data['DEATH_EVENT'].value_counts()

# Summary statistics
print("Summary Statistics:")
heart_failure_data.describe()

# Check for missing values
missing_values = heart_failure_data.isnull().sum()

# Convert to a DataFrame for better visualization
missing_values_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Values': missing_values.values
})

# Display the DataFrame
missing_values_df

aly.heatmap(heart_failure_data,color="DEATH_EVENT")


# Distributions of all columns
print("Visualizing distributions for all columns...")
aly.dist(heart_failure_data)


aly.pair(heart_failure_data,color="DEATH_EVENT")

aly.corr(data.movies())

aly.parcoord(heart_failure_data,color = 'DEATH_EVENT')

# Create the distribution plots
aly.dist(heart_failure_data,color = 'DEATH_EVENT')

#Data Splitting!

heart_failure_data = pd.read_csv(file_path)

heart_failure_train, heart_failure_test = train_test_split(heart_failure_data, 
                                                           train_size = 0.8, 
                                                           stratify = heart_failure_data['DEATH_EVENT'],
                                                           random_state = 522)

url_processed = '../data/processed/'
heart_failure_train.to_csv(os.path.join(url_processed, 'heart_failure_train.csv'))
heart_failure_test.to_csv(os.path.join(url_processed, 'heart_failure_test.csv'))

#Preprocessing Columns!

# Define numeric columns
numeric_columns = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 
                   'platelets', 'serum_creatinine', 'serum_sodium', 'time']
# List of binary columns
binary_columns = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

# Convert all binary columns to True/False so they're treated as categorical data
heart_failure_train[binary_columns] = heart_failure_train[binary_columns].astype(bool)
heart_failure_test[binary_columns] = heart_failure_test[binary_columns].astype(bool)



preprocessor = make_column_transformer(
    (StandardScaler(), numeric_columns),
    (OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop='if_binary', dtype = int), binary_columns),
    remainder = 'passthrough'
)

preprocessor.fit(heart_failure_train)
heart_failure_scaled_train = preprocessor.transform(heart_failure_train)
heart_failure_scaled_test = preprocessor.transform(heart_failure_test)

preprocessor.verbose_feature_names_out = False
column_names = (preprocessor.get_feature_names_out().tolist())
scaled_train = pd.DataFrame(heart_failure_scaled_train, columns=column_names)

#Correlation matrix part starts from here:
correlation_matrix = scaled_train.drop(columns=['DEATH_EVENT']).corr()
correlation_long = correlation_matrix.reset_index().melt(id_vars='index')
correlation_long.columns = ['Feature 1', 'Feature 2', 'Correlation']

correlation_heatmap = alt.Chart(correlation_long).mark_rect().encode(
    x='Feature 1:O',
    y='Feature 2:O',
    color=alt.Color('Correlation:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['Feature 1', 'Feature 2', 'Correlation']
).properties(
    width=600,
    height=600,
    title="Correlation Heatmap"
)


# Specify the folder path and save the chart
folder_path = '../results/figures'  
correlation_heatmap.save(f'{folder_path}/correlation_heatmap.png')


correlation_heatmap

#Based on the correlation matrix graph below, all features have relatively low correlations between each other, 
#the correlations are below 0.5, so there is no strong evidence to drop additional featues. 

# validate training data for anomalous correlations between target/response variable 
# and features/explanatory variables, 
# as well as anomalous correlations between features/explanatory variables
# Do these on training data as part of EDA! 

warnings.filterwarnings("ignore", category=FutureWarning, module="deepchecks")

scaled_train_ds = Dataset(scaled_train, label="DEATH_EVENT", cat_features=[])

check_feat_lab_corr = FeatureLabelCorrelation().add_condition_feature_pps_less_than(0.9)
check_feat_lab_corr_result = check_feat_lab_corr.run(dataset=scaled_train_ds)

check_feat_feat_corr = FeatureFeatureCorrelation().add_condition_max_number_of_pairs_above_threshold(threshold = 0.92, n_pairs = 0)
check_feat_feat_corr_result = check_feat_feat_corr.run(dataset=scaled_train_ds)

if not check_feat_lab_corr_result.passed_conditions():
    raise ValueError("Feature-Label correlation exceeds the maximum acceptable threshold.")

if not check_feat_feat_corr_result.passed_conditions():
    raise ValueError("Feature-feature correlation exceeds the maximum acceptable threshold.")

#Building the model!

#Decision Tree
pipeline = make_pipeline(
        preprocessor, 
        DecisionTreeClassifier(random_state=522)
    )

dt_scores = cross_validate(pipeline, 
                           heart_failure_train.drop(columns=['DEATH_EVENT']), 
                           heart_failure_train['DEATH_EVENT'],
                           return_train_score=True
                          )

dt_scores = pd.DataFrame(dt_scores).sort_values('test_score', ascending = False)
dt_scores


#KNN
pipeline = make_pipeline(
        preprocessor, 
        KNeighborsClassifier()
    )

param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 100, 3)
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,  
    n_jobs=-1,  
    return_train_score=True,
)

heart_failure_fit = grid_search.fit(heart_failure_train.drop(columns=['DEATH_EVENT']), heart_failure_train['DEATH_EVENT'] )

knn_best_model = grid_search.best_estimator_ 
knn_best_model

pd.DataFrame(grid_search.cv_results_).sort_values('mean_test_score', ascending = False)[['params', 'mean_test_score']].iloc[0]

#Logistic Regression

pipeline = make_pipeline(
        preprocessor, 
        LogisticRegression(random_state=522, max_iter=2000, class_weight = "balanced")
    )

param_grid = {
    "logisticregression__C": 10.0 ** np.arange(-5, 5, 1)
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,  
    n_jobs=-1,  
    return_train_score=True
)

heart_failure_fit = grid_search.fit(heart_failure_train.drop(columns=['DEATH_EVENT']), heart_failure_train['DEATH_EVENT'] )

lr_best_model = grid_search.best_estimator_.named_steps['logisticregression']
lr_best_model

lr_scores = pd.DataFrame(grid_search.cv_results_).sort_values('mean_test_score', ascending = False)[['param_logisticregression__C', 'mean_test_score', 'mean_train_score']]
lr_scores.iloc[0:5]

# Log scale for x-axis, fixed y-axis range, and explicit data type specification
lr_train_test_cv = alt.Chart(lr_scores).transform_fold(
    ["mean_test_score", "mean_train_score"],  # Combine columns into one for color differentiation
    as_=["Score Type", "Score"]  # Rename columns for legend and y-axis
).mark_line().encode(
    x=alt.X("param_logisticregression__C:Q", 
            title="C (Regularization Parameter)", 
            scale=alt.Scale(type='log')),  # Set x-axis to log scale
    y=alt.Y("Score:Q", 
            title="Score", 
            scale=alt.Scale(domain=[0.75, 0.85])),  # Set y-axis range
    color=alt.Color("Score Type:N", 
                    title="Score Type",  # Add legend title
                    scale=alt.Scale(domain=["mean_test_score", "mean_train_score"],
                                    range=["skyblue", "pink"])),  # Map colors to lines
    tooltip=["param_logisticregression__C", "Score Type:N", "Score:Q"]  # Explicitly specify data types in tooltip
).properties(
    title="Training vs Cross-Validation Scores (Log Scale)",
    width=600,
    height=400
)

# Specify the folder path and save the chart
folder_path = '../results/figures'  
lr_train_test_cv.save(f'{folder_path}/lr_cv_scores.png')

lr_train_test_cv

features = lr_best_model.coef_
feature_names = heart_failure_train.drop(columns=['DEATH_EVENT']).columns
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': features[0],
    'Absolute_Coefficient': abs(features[0])
}).sort_values(by='Absolute_Coefficient', ascending=False)

coefficients

#Model Evaluation!

#Confusion Matrix

# Confusion Matrix

heart_failure_predictions = heart_failure_test.assign(
    predicted=heart_failure_fit.predict(heart_failure_test)
)

cm_crosstab = pd.crosstab(heart_failure_predictions['DEATH_EVENT'], 
                          heart_failure_predictions['predicted'], 
                          rownames=["Actual"], 
                          colnames=["Predicted"]
                         )


cm_crosstab
# cm = confusion_matrix(heart_failure_test["DEATH_EVENT"], heart_failure_fit.predict(heart_failure_test))
# cm

accuracy = round(accuracy_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted']),2)
precision = round(precision_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted']),2)
recall = round(recall_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted']),2)
f1 = round(f1_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted']),2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
#| label: tbl-confusion-matrix
#| tbl-cap: Confusion matrix for the final model on the test dataset.
#| echo: false
#| output: false

import pandas as pd

# Generate confusion matrix as a crosstab
heart_failure_predictions2 = heart_failure_test.assign(
    predicted=heart_failure_fit.predict(heart_failure_test)
)

cm_crosstab = pd.crosstab(
    heart_failure_predictions['DEATH_EVENT'], 
    heart_failure_predictions['predicted'], 
    rownames=["Actual"], 
    colnames=["Predicted"]
)
TP = cm_crosstab.iloc[1,1]
TN = cm_crosstab.iloc[0,0]
FP = cm_crosstab.iloc[0,1]
FN = cm_crosstab.iloc[1,0]

# Render the confusion matrix as it is for correct alignment
cm_crosstab.style.set_table_attributes("style='display:inline'")

## Summary

We built a classification model using the logistic regression algorithm to predict survival outcomes for patients with heart failure. Using patient test results, the final classifier achieves an accuracy of `{python} accuracy`. The model’s precision of `{python} precision` suggests it is moderately conservative in predicting the positive class (death), minimizing false alarms. More importantly, the recall of `{python} recall` ensures the model identifies the majority of high-risk patients, reducing the likelihood of missing true positive cases, which could have serious consequences. The F1-score of `{python} f1` reflects a good balance between precision and recall, highlighting the model’s robustness in survival prediction, see @tbl-model-metrics2.

From the confusion matrix, the model correctly identified `{python} TP` patients who passed away (true positives) and`{python} TN` patients who survived (true negatives). However, it also predicted `{python} FP` false positives, incorrectly classifying some survivors as deceased, and missed `{python} FN` actual cases of death (false negatives). While these errors warrant consideration, the model’s performance demonstrates strong predictive capabilities for both positive and negative outcomes, see @tbl-confusion-matrix2.

Overall, the logistic regression classifier effectively leverages patient test results to support survival prediction, providing a valuable tool to aid clinical decision-making in heart failure management.

## Introduction

Cardiovascular diseases are responsible for approximately 17 million deaths globally each year, with heart failure and myocardial infarctions being the leading contributors to this staggering toll [@chicco2020]. Electronic medical records from patients with heart failure, collected during follow-up care, provide a wealth of data on symptoms, test results, and clinical outcomes. Leveraging this data, our team applies machine learning algorithms to predict patient survival after heart failure. This approach uncovers critical patterns and insights that might otherwise remain hidden from traditional clinical assessments, offering valuable tools to support medical decision-making and improve patient outcomes. 

## Data 

We analyzed a dataset containing the medical records of 299 heart failure patients [@dua2017]. The patients consisted of 105 women and 194 men, and their ages range between 40 and 95 years old. The dataset contains 13 features shown in @tbl-patient-table, which report clinical, body, and lifestyle information [@heartfailuredata]. The **death event** was used as the target variable in our binary classification study. It states whether the patient died or survived before the end of the follow-up period, which lasted 130 days on average. Our dataset has a class imbalance where the number of survived patients (death event = 0) is 203 (67.89%) and the number of dead patients (death event = 1) is 96 (32.11%), see @tbl-death-event-counts.



## Explanatory Data Analysis and Visualizations


In [ ]:
#| label: tbl-patient-table
#| tbl-cap: Description of the columns in the heart failure dataset.
#| echo: false

import pandas as pd

# Load the table
patient_table = pd.read_csv("../results/tables/patient_table.csv")

# Apply CSS to left-align all columns
patient_table.style.set_properties(**{'text-align': 'left'})

![Correlation heatmap-  Based on the correlation matrix graph below, all features have relatively low correlations between each other, the correlations are below 0.5, so there is no strong evidence to drop additional featues. ](../results/figures/correlation_heatmap.png){#correlation_heatmap width=60% fig-pos="H"}


In [ ]:
#| label: tbl-missing-values
#| tbl-cap: Number of missing values in each column of the heart failure dataset.
#| echo: false

file_path = '../data/heart_failure_clinical_records_dataset.csv'
heart_failure_data = pd.read_csv(file_path)

# Check for missing values
missing_values = heart_failure_data.isnull().sum()

# Convert to a DataFrame for better visualization
missing_values_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Values': missing_values.values
})

# Style the DataFrame to align text to the left
missing_values_df.style.set_properties(**{'text-align': 'left'})

No missing values, no imputation or filling Nulls required


In [ ]:
#| label: tbl-death-event-counts
#| tbl-cap: Distribution of the target variable `DEATH_EVENT` in the heart failure dataset.
#| echo: false

import pandas as pd

# Load the dataset
file_path = '../data/heart_failure_clinical_records_dataset.csv'
heart_failure_data = pd.read_csv(file_path)

# Get value counts for DEATH_EVENT and convert to a DataFrame
death_event_counts = heart_failure_data['DEATH_EVENT'].value_counts().reset_index()
death_event_counts.columns = ['DEATH_EVENT', 'Count']  # Rename columns

# Display the DataFrame as a table
death_event_counts

Dataset Size: The dataset is relatively small, with only 300 rows. \
Class Imbalance: The target variable, DEATH_EVENT, has few examples in the "True" class (i.e., the event occurred), which might affect the model's ability to learn and generalize well. This class imbalance will be taken into consideration during analysis and model evaluation.



## Model

We compared Decision Tree, KNN, Logistic Regression, and selected Logistic Regression due to its interpretability, and ability to handle both linear and non-linear relationships between features. Logistic Regression performed better than the other two models as it works well with fewer features and is less prone to overfitting compared to more complex models like Decision Trees or KNN, especially when the data is relatively small.


In [ ]:
heart_failure_data = pd.read_csv(file_path)

heart_failure_train, heart_failure_test = train_test_split(heart_failure_data, 
                                                           train_size = 0.8, 
                                                           stratify = heart_failure_data['DEATH_EVENT'],
                                                           random_state = 522)

url_processed = '../data/processed/'
heart_failure_train.to_csv(os.path.join(url_processed, 'heart_failure_train.csv'))
heart_failure_test.to_csv(os.path.join(url_processed, 'heart_failure_test.csv'))

# Define numeric columns
numeric_columns = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 
                   'platelets', 'serum_creatinine', 'serum_sodium', 'time']
# List of binary columns
binary_columns = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

# Convert all binary columns to True/False so they're treated as categorical data
heart_failure_train[binary_columns] = heart_failure_train[binary_columns].astype(bool)
heart_failure_test[binary_columns] = heart_failure_test[binary_columns].astype(bool)

preprocessor = make_column_transformer(
    (StandardScaler(), numeric_columns),
    (OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop='if_binary', dtype = int), binary_columns),
    remainder = 'passthrough'
)

In [ ]:
#| echo: false
#| output: false

pipeline = make_pipeline(
        preprocessor, 
        LogisticRegression(random_state=522, max_iter=2000, class_weight = "balanced")
    )

param_grid = {
    "logisticregression__C": 10.0 ** np.arange(-5, 5, 1)
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,  
    n_jobs=-1,  
    return_train_score=True
)

heart_failure_fit = grid_search.fit(heart_failure_train.drop(columns=['DEATH_EVENT']), heart_failure_train['DEATH_EVENT'] )

lr_best_model = grid_search.best_estimator_.named_steps['logisticregression']
lr_best_model

Hyperparameter tuning to find find the best Logistic Regression model:

In [ ]:
lr_scores = pd.DataFrame(grid_search.cv_results_).sort_values('mean_test_score', ascending = False)[['param_logisticregression__C', 'mean_test_score', 'mean_train_score']]
logregC = lr_scores.iloc[0,0]
logreg_cv = round(lr_scores.iloc[0,1],2)
lr_scores.iloc[0:5]

**The model is performing well with C = `{python} logregC` with a CV score of `{python} logreg_cv` and is close to train score, indicating that model is generalising well.**

![Cross-validation scores for Logistic Regression -  Logistic regression performs better than Decision tree and KNN on the cross validation data, hence, we selected it as our final model.](../results/figures/lr_cv_scores.png){#fig-lr_cv_scores width=60% fig-pos="H"}




The best features to train our model are show in @tbl-top-features:


In [ ]:
#| label: tbl-top-features
#| tbl-cap: Top features for trainig the model.
#| echo: false

features = lr_best_model.coef_
feature_names = heart_failure_train.drop(columns=['DEATH_EVENT']).columns
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': features[0],
    'Absolute_Coefficient': abs(features[0])
}).sort_values(by='Absolute_Coefficient', ascending=False)

coefficients

## Model Evaluation

#### Confusion Matrix


In [ ]:
#| label: tbl-confusion-matrix2
#| tbl-cap: Confusion matrix for the final model on the test dataset.
#| echo: false

import pandas as pd

# Generate confusion matrix as a crosstab
heart_failure_predictions = heart_failure_test.assign(
    predicted=heart_failure_fit.predict(heart_failure_test)
)

cm_crosstab2 = pd.crosstab(
    heart_failure_predictions['DEATH_EVENT'], 
    heart_failure_predictions['predicted'], 
    rownames=["Actual"], 
    colnames=["Predicted"]
)
TP2 = cm_crosstab.iloc[1,1]
TN2 = cm_crosstab.iloc[0,0]
FP2 = cm_crosstab.iloc[0,1]
FN2 = cm_crosstab.iloc[1,0]

# Render the confusion matrix as it is for correct alignment
cm_crosstab.style.set_table_attributes("style='display:inline'")

In [ ]:
#| echo: false
#| label: tbl-model-metrics2
#| tbl-cap: Evaluation metrics for the final model.

import pandas as pd

# Calculate evaluation metrics
accuracy2 = accuracy_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted'])
precision2 = precision_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted'])
recall2 = recall_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted'])
f1_2 = f1_score(heart_failure_predictions['DEATH_EVENT'], heart_failure_predictions['predicted'])

# Create a DataFrame for the metrics
metrics_table2 = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1]
})

# Round values for better readability
metrics_table2['Value'] = metrics_table2['Value'].round(4)

# Display the DataFrame
metrics_table2

## Results and Conculsion

The analysis revealed that `platelets` and `ejection_fraction` are the most important features (see @tbl-top-features) in predicting the risk of patient mortality. These features significantly impact the model's ability to assess patient risk, which is crucial for early intervention. Our model achieved a recall score of `{python} recall` (see @tbl-model-metrics2), which is a good start, but there is room for improvement, particularly in reducing the number of high risk patients the model might miss, i.e., maximising recall by minimising False Negatives.

The main challenges in this project stem from class imbalance and limited data availability. With more diverse and comprehensive datasets, performance could be further enhanced. We would also like to explore other machine learning models to improve the overall accuracy.

In conclusion, while the current model shows potential, there is significant opportunity to enhance its effectiveness. With improvements in data quality and model optimization, this tool could become a crucial asset in predicting patient risk and saving lives.


## References
